In [1]:
import pandas as pd
import statsmodels.api as sm

# Load your data from Excel
df = pd.read_excel('climate_final_with_ff5_and_excess_return.xlsx', sheet_name='climate_final_with_ff5_and_exce', parse_dates=['date'])

# Define your factor lists
fama_french_factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']
env_factors = ['Overall_ESG_Exposure', 'Overall_ESG_Sentiment']

# Helper function to run regressions and collect results
def run_regression(y, X):
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    results = {}
    for var in X.columns:
        results[f'{var}_coef'] = model.params[var]
        results[f'{var}_stderr'] = model.bse[var]  
    results['R2'] = model.rsquared
    results['Adj_R2'] = model.rsquared_adj
    results['n_obs']   = int(model.nobs)
    return results

# --- 1. Macro Sector Level Regression ---
sector_results_fama = []
sector_results_fama_env = []

for sector, group in df.groupby('Macro_Sector'):
    # Fama-French only
    res_fama = run_regression(group['excess_RET'], group[fama_french_factors])
    res_fama['Macro_Sector'] = sector
    sector_results_fama.append(res_fama)
    
    # Fama-French + ENV
    res_fama_env = run_regression(group['excess_RET'], group[fama_french_factors + env_factors])
    res_fama_env['Macro_Sector'] = sector
    sector_results_fama_env.append(res_fama_env)

sector_df_fama = pd.DataFrame(sector_results_fama)
sector_df_fama_env = pd.DataFrame(sector_results_fama_env)

# --- 2. Ticker Level Regression (within each Macro Sector) ---
ticker_results = []

ticker_groups = df.groupby(['Macro_Sector', 'TICKER'])

for (sector, ticker), group in ticker_groups:
    res = run_regression(group['excess_RET'], group[fama_french_factors + env_factors])
    res['Macro_Sector'] = sector
    res['Ticker'] = ticker
    ticker_results.append(res)

full_ticker_df = pd.DataFrame(ticker_results)

# --- 3. Pretty-printing ---
# Split ticker results by sector
industrials = full_ticker_df[full_ticker_df['Macro_Sector'] == 'Industrials']
consumer = full_ticker_df[full_ticker_df['Macro_Sector'] == 'Consumer']
energy_utilities = full_ticker_df[full_ticker_df['Macro_Sector'] == 'Energy & Utilities']

# Print results
print("\n=== Macro-Sector Level Regression: Fama-French Only ===")
print(sector_df_fama.round(4))

print("\n=== Macro-Sector Level Regression: Fama-French + ENV Factors ===")
print(sector_df_fama_env.round(4))

print("\n=== Ticker-Level Regression: Industrials ===")
print(industrials.round(4))

print("\n=== Ticker-Level Regression: Consumer ===")
print(consumer.round(4))

print("\n=== Ticker-Level Regression: Energy & Utilities ===")
print(energy_utilities.round(4))

print("\nDone! All tables printed.")



=== Macro-Sector Level Regression: Fama-French Only ===
   const_coef  const_stderr  Mkt-RF_coef  Mkt-RF_stderr  SMB_coef  SMB_stderr  \
0     -0.0034        0.0023       1.0880         0.0478    0.2473      0.0939   
1     -0.0033        0.0023       0.7781         0.0481   -0.1016      0.0949   
2     -0.0028        0.0017       0.9543         0.0349    0.1912      0.0690   

   HML_coef  HML_stderr  RMW_coef  RMW_stderr  CMA_coef  CMA_stderr      R2  \
0    0.0279      0.0793    0.2349      0.1078   -0.0072      0.1126  0.3478   
1    0.2424      0.0797    0.1790      0.1085    0.3939      0.1132  0.2976   
2    0.1192      0.0579    0.2319      0.0789    0.1330      0.0823  0.3019   

   Adj_R2  n_obs        Macro_Sector  
0  0.3455   1412            Consumer  
1  0.2937    910  Energy & Utilities  
2  0.3005   2520         Industrials  

=== Macro-Sector Level Regression: Fama-French + ENV Factors ===
   const_coef  const_stderr  Mkt-RF_coef  Mkt-RF_stderr  SMB_coef  SMB_stderr  

In [5]:
import pandas as pd
import statsmodels.api as sm

# Load your data from Excel
df = pd.read_excel(
    'climate_final_with_ff5_and_excess_return.xlsx',
    sheet_name='climate_final_with_ff5_and_exce',
    parse_dates=['date']
)

# Define your factor lists
fama_french_factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']
env_factors = ['Overall_ESG_Exposure', 'Overall_ESG_Sentiment']

# Helper function to run regressions and collect results
def run_regression(y, X):
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    results = {}
    for var in X.columns:
        results[f'{var}_coef']   = model.params[var]
        results[f'{var}_tstat']  = model.tvalues[var]
        results[f'{var}_stderr'] = model.bse[var]
    results['R2']     = model.rsquared
    results['Adj_R2'] = model.rsquared_adj
    return results

# --- 1. Macro‐Sector Level Regressions ---
sector_results_fama     = []
sector_results_fama_env = []

for sector, group in df.groupby('Macro_Sector'):
    # Fama‐French only
    res_fama = run_regression(group['excess_RET'], group[fama_french_factors])
    res_fama['Macro_Sector'] = sector
    sector_results_fama.append(res_fama)
    # Fama‐French + ENV
    res_fama_env = run_regression(
        group['excess_RET'],
        group[fama_french_factors + env_factors]
    )
    res_fama_env['Macro_Sector'] = sector
    sector_results_fama_env.append(res_fama_env)

sector_df_fama     = pd.DataFrame(sector_results_fama)
sector_df_fama_env = pd.DataFrame(sector_results_fama_env)

# --- 2. Ticker‐Level Regressions (within each Macro Sector) ---
ticker_results_fama     = []
ticker_results_fama_env = []

for (sector, ticker), group in df.groupby(['Macro_Sector', 'TICKER']):
    # Fama‐French only
    res_t_fama = run_regression(group['excess_RET'], group[fama_french_factors])
    res_t_fama['Macro_Sector'] = sector
    res_t_fama['Ticker']       = ticker
    ticker_results_fama.append(res_t_fama)
    # Fama‐French + ENV
    res_t_fama_env = run_regression(
        group['excess_RET'],
        group[fama_french_factors + env_factors]
    )
    res_t_fama_env['Macro_Sector'] = sector
    res_t_fama_env['Ticker']       = ticker
    ticker_results_fama_env.append(res_t_fama_env)

ticker_df_fama     = pd.DataFrame(ticker_results_fama)
ticker_df_fama_env = pd.DataFrame(ticker_results_fama_env)

# --- 3. Pretty‐printing ---
print("\n=== Macro‐Sector Level: FF5 Only ===")
print(sector_df_fama.round(4))

print("\n=== Macro‐Sector Level: FF5 + ENV ===")
print(sector_df_fama_env.round(4))

for name, df_t in [
    ("Industrials", ticker_df_fama[ticker_df_fama['Macro_Sector']=='Industrials']),
    ("Consumer",    ticker_df_fama[ticker_df_fama['Macro_Sector']=='Consumer']),
    ("Energy & Utilities", ticker_df_fama[ticker_df_fama['Macro_Sector']=='Energy & Utilities'])
]:
    print(f"\n=== Ticker‐Level ({name}): FF5 Only ===")
    print(df_t.round(4))

for name, df_t in [
    ("Industrials", ticker_df_fama_env[ticker_df_fama_env['Macro_Sector']=='Industrials']),
    ("Consumer",    ticker_df_fama_env[ticker_df_fama_env['Macro_Sector']=='Consumer']),
    ("Energy & Utilities", ticker_df_fama_env[ticker_df_fama_env['Macro_Sector']=='Energy & Utilities'])
]:
    print(f"\n=== Ticker‐Level ({name}): FF5 + ENV ===")
    print(df_t.round(4))

print("\nDone! All tables printed.")


=== Macro‐Sector Level: FF5 Only ===
   const_coef  const_tstat  const_stderr  Mkt-RF_coef  Mkt-RF_tstat  \
0     -0.0034      -1.4755        0.0023       1.0880       22.7770   
1     -0.0033      -1.3986        0.0023       0.7781       16.1851   
2     -0.0028      -1.6424        0.0017       0.9543       27.3059   

   Mkt-RF_stderr  SMB_coef  SMB_tstat  SMB_stderr  HML_coef  ...  HML_stderr  \
0         0.0478    0.2473     2.6322      0.0939    0.0279  ...      0.0793   
1         0.0481   -0.1016    -1.0700      0.0949    0.2424  ...      0.0797   
2         0.0349    0.1912     2.7711      0.0690    0.1192  ...      0.0579   

   RMW_coef  RMW_tstat  RMW_stderr  CMA_coef  CMA_tstat  CMA_stderr      R2  \
0    0.2349     2.1803      0.1078   -0.0072    -0.0639      0.1126  0.3478   
1    0.1790     1.6497      0.1085    0.3939     3.4795      0.1132  0.2976   
2    0.2319     2.9392      0.0789    0.1330     1.6157      0.0823  0.3019   

   Adj_R2        Macro_Sector  
0  0.34

In [7]:
import pandas as pd
import statsmodels.api as sm

# === Load data ===
df = pd.read_excel(
    'climate_final_with_ff5_and_excess_return.xlsx',
    sheet_name='climate_final_with_ff5_and_exce',
    parse_dates=['date']
)

# === Define factor sets ===
fama_french_factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']
env_factors = ['Overall_ESG_Exposure', 'Overall_ESG_Sentiment']

# === Filter to only the tickers shown in the LaTeX tables ===
selected_tickers = [
    # Panel A: Industrials
    'ZTS', 'PEP', 'ANET', 'JNJ', 'CMI', 'SWK', 'BSX', 'RCL', 'EL',
    # Panel B: Consumer
    'CDNS', 'ORLY', 'IT', 'AAP',
    # Panel C: Energy & Utilities
    'WMB', 'AES'
]
df = df[df['TICKER'].isin(selected_tickers)]

# === Helper function to run regressions and extract stats ===
def run_regression(y, X):
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    results = {}
    for var in X.columns:
        results[f'{var}_coef']   = model.params[var]
        results[f'{var}_tstat']  = model.tvalues[var]
        results[f'{var}_stderr'] = model.bse[var]
    results['R2']     = model.rsquared
    results['Adj_R2'] = model.rsquared_adj
    return results

# === 1. Macro‐Sector Level Regressions ===
sector_results_fama     = []
sector_results_fama_env = []

for sector, group in df.groupby('Macro_Sector'):
    # FF5 only
    res_fama = run_regression(group['excess_RET'], group[fama_french_factors])
    res_fama['Macro_Sector'] = sector
    sector_results_fama.append(res_fama)
    # FF5 + ENV
    res_fama_env = run_regression(group['excess_RET'], group[fama_french_factors + env_factors])
    res_fama_env['Macro_Sector'] = sector
    sector_results_fama_env.append(res_fama_env)

sector_df_fama     = pd.DataFrame(sector_results_fama)
sector_df_fama_env = pd.DataFrame(sector_results_fama_env)

# === 2. Ticker‐Level Regressions (Filtered Tickers Only) ===
ticker_results_fama     = []
ticker_results_fama_env = []

for (sector, ticker), group in df.groupby(['Macro_Sector', 'TICKER']):
    # FF5 only
    res_t_fama = run_regression(group['excess_RET'], group[fama_french_factors])
    res_t_fama['Macro_Sector'] = sector
    res_t_fama['Ticker']       = ticker
    ticker_results_fama.append(res_t_fama)
    # FF5 + ENV
    res_t_fama_env = run_regression(group['excess_RET'], group[fama_french_factors + env_factors])
    res_t_fama_env['Macro_Sector'] = sector
    res_t_fama_env['Ticker']       = ticker
    ticker_results_fama_env.append(res_t_fama_env)

ticker_df_fama     = pd.DataFrame(ticker_results_fama)
ticker_df_fama_env = pd.DataFrame(ticker_results_fama_env)

# === 3. Print Results ===
print("\n=== Macro‐Sector Level: FF5 Only ===")
print(sector_df_fama.round(4))

print("\n=== Macro‐Sector Level: FF5 + ENV ===")
print(sector_df_fama_env.round(4))

for name, df_t in [
    ("Industrials", ticker_df_fama[ticker_df_fama['Macro_Sector']=='Industrials']),
    ("Consumer",    ticker_df_fama[ticker_df_fama['Macro_Sector']=='Consumer']),
    ("Energy & Utilities", ticker_df_fama[ticker_df_fama['Macro_Sector']=='Energy & Utilities'])
]:
    print(f"\n=== Ticker‐Level ({name}): FF5 Only ===")
    print(df_t.round(4))

for name, df_t in [
    ("Industrials", ticker_df_fama_env[ticker_df_fama_env['Macro_Sector']=='Industrials']),
    ("Consumer",    ticker_df_fama_env[ticker_df_fama_env['Macro_Sector']=='Consumer']),
    ("Energy & Utilities", ticker_df_fama_env[ticker_df_fama_env['Macro_Sector']=='Energy & Utilities'])
]:
    print(f"\n=== Ticker‐Level ({name}): FF5 + ENV ===")
    print(df_t.round(4))

print("\nDone! All tables printed.")


=== Macro‐Sector Level: FF5 Only ===
   const_coef  const_tstat  const_stderr  Mkt-RF_coef  Mkt-RF_tstat  \
0     -0.0003      -0.0586        0.0051       1.0003        9.4360   
1     -0.0013      -0.1925        0.0070       1.0616        7.3924   
2     -0.0027      -0.7935        0.0034       0.9948       14.1435   

   Mkt-RF_stderr  SMB_coef  SMB_tstat  SMB_stderr  HML_coef  ...  HML_stderr  \
0         0.1060    0.0902     0.4391      0.2055    0.1744  ...      0.1768   
1         0.1436    0.0304     0.1073      0.2835    0.3028  ...      0.2381   
2         0.0703    0.1688     1.2159      0.1389    0.0401  ...      0.1166   

   RMW_coef  RMW_tstat  RMW_stderr  CMA_coef  CMA_tstat  CMA_stderr      R2  \
0    0.2779     1.1655      0.2384   -0.1855    -0.7387      0.2511  0.3118   
1   -0.1168    -0.3604      0.3242    0.6640     1.9635      0.3382  0.3798   
2    0.3859     2.4301      0.1588    0.1365     0.8241      0.1656  0.3134   

   Adj_R2        Macro_Sector  
0  0.29